In [1]:
# Hello there.   This is a do-over of the floating preferred analyzer

# Time to develop clean modules. Let's start with reading in a list of prefs
# The input data consists of three separate tables:
# a) A listing of tickers and their configurations
# b) A list of ratings for each company (equivalently, of the ticker)
# c) The current interest rate environment, which has only the 3-month rate and the prime rate



In [4]:
# Imports go here
import pandas as pd
import requests_cache
# local ones
import prefcode as pc


In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
# Set up cache
session_cached = requests_cache.CachedSession('yfinance.cache',expire_after=3600)
session_uncached = requests_cache.CachedSession('yfinance2.cache',expire_after=30)


In [26]:
# Read the prefs database

print("Setting up databases")
df, interest_db = pc.setup_databases()
interest_db = {
    'tbill': 1.37, 'goc5': 2.75, 'prime': 3.2}

print(interest_db)

print("Fetching price data")
df = pc.fetch_prices(df, session_uncached, fetch=True)
#df.head(10).sort_values(by='Price', ascending=True)

Setting up databases
{'tbill': 1.37, 'goc5': 2.75, 'prime': 3.2}
Fetching price data


In [27]:
print("Updating dividends and current yield")
df = pc.update_div_and_yield(df, interest_db,price_column="Price")

Updating dividends and current yield


In [28]:
print("Updating market spread for later calculation")
df2 = pc.update_market_spread(df, interest_db)

print("Dropping prime-related preferreds. Maybe another day")
tdf = df[df['Type'] == 'T'].copy()

print("Calculating scenarios for T-bill rate")
scenarios = {"137":  [1.37,  0.20],
             "185":  [1.85,  0.40],
             "210":  [2.10,  0.30],
             "225":  [2.25,  0.10]}

pc.update_expected_yield(tdf, scenarios)

Updating market spread for later calculation
Dropping prime-related preferreds. Maybe another day
Calculating scenarios for T-bill rate


In [25]:
print("Here are the highest yield items")
tdf.sort_values(by='ExpYield', ascending=False).head(15)

Here are the highest yield items


,Ticker,Spread,Type,Mult,Rating,Price,AnnualDiv,CurYieldPct,MSpread,137_Yield,185_Yield,210_Yield,225_Yield,ExpYield
30,TRP.PR.H,128.0,T,NaN,P2L,14.45,0.6625,4.5848,3.2148,4.5842,11.5036,14.6124,16.3412,11.53612
20,FN.PR.B,207.0,T,NaN,P3I,14.76,0.8600,5.8266,4.4566,5.8258,11.1062,13.5502,14.9291,11.16561
14,CVE.PR.B,173.0,T,NaN,P2L,15.30,0.7750,5.0654,3.6954,5.0646,10.5523,13.0505,14.4485,10.59384
31,TRP.PR.I,154.0,T,NaN,P2L,15.45,0.7275,4.7087,3.3387,4.7095,10.4276,13.0061,14.4424,10.45901
21,FTS.PR.I,145.0,T,NaN,P3H,15.68,0.7050,4.4962,3.1262,4.4956,10.2291,12.7981,14.2249,10.25268
22,MFC.PR.P,141.0,T,NaN,P2I,15.75,0.6950,4.4127,3.0427,4.4127,10.1744,12.7493,14.1775,10.19484
25,SLF.PR.J,141.0,T,NaN,P2H,15.78,0.6950,4.4043,3.0343,4.4045,10.1465,12.7119,14.1346,10.16653
24,PWF.PR.Q,160.0,T,NaN,P2H,16.05,0.7425,4.6262,3.2562,4.6255,9.8675,12.2256,13.5377,9.89355
27,TA.PR.E,203.0,T,NaN,P3L,16.25,0.8500,5.2308,3.8608,5.2302,9.7561,11.8246,12.9845,9.79431
17,FFH.PR.F,216.0,T,NaN,P3H,16.61,0.8825,5.3131,3.9431,5.3124,9.4977,11.4142,12.4899,9.53481


In [11]:
print("And uniquified by issuer")
# Uniquify by parent
tdf['Parent'] = [x.split('.')[0] for x in tdf['Ticker']]
#tdf
uniq_parent = pc.summarize_best_by_column(tdf,'ExpYield','Parent')

uniq_parent = uniq_parent.reindex(columns=["Ticker", "Rating", 
                                           "Spread","CurYieldPct","ExpYield"])
# make it a bit more readable

uniq_parent['CurYield'] = [round(x,3) for x in uniq_parent['CurYieldPct']]
uniq_parent['ExpYield'] = [round(x,3) for x in uniq_parent['ExpYield']]
uniq_parent.drop(columns=['CurYieldPct'], errors='ignore', inplace=True)
uniq_parent.sort_values(by='ExpYield', ascending=False).head(15)


And uniquified by issuer


,Ticker,Rating,Spread,ExpYield,CurYield
30,TRP.PR.H,P2L,128.0,21.252,3.229
20,FN.PR.B,P3I,207.0,17.517,4.469
24,PWF.PR.Q,P2H,160.0,16.003,3.333
21,FTS.PR.I,P3H,145.0,15.996,3.058
25,SLF.PR.J,P2H,141.0,15.989,2.984
22,MFC.PR.P,P2I,141.0,15.333,2.924
14,CVE.PR.B,P2L,173.0,15.305,3.491
27,TA.PR.E,P3L,203.0,15.203,4.020
15,EMA.PR.B,P3H,184.0,14.181,3.548
17,FFH.PR.F,P3H,216.0,13.076,3.930


In [59]:
best_scn_df = pc.summarize_best_by_column(tdf, 'ExpYield')
best_scn_df = best_scn_df.reindex(columns=["Ticker", "Rating", "Spread","ExpYield", "MSpread"])

print("And here are the best by rating level")
best_scn_df.sort_values(by=['Rating'])

And here are the best by rating level


,Ticker,Rating,Spread,ExpYield,MSpread
24,PWF.PR.Q,P2H,160.0,16.178195,2.7890
22,MFC.PR.P,P2I,141.0,14.080700,2.2014
30,TRP.PR.H,P2L,128.0,19.951310,2.4411
21,FTS.PR.I,P3H,145.0,13.936625,2.2648
20,FN.PR.B,P3I,207.0,15.063640,3.6353
27,TA.PR.E,P3L,203.0,12.642965,3.1718


In [60]:
# Now, check market spread and flag if below average for the group
# That could result in a re-rating
#mdf = pc.calculate_avg_per_rating(tdf, column='MSpread')
#mdf